In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import cross_validation
%matplotlib inline
import gensim
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.ldamodel import LdaModel

In [4]:
master = pd.read_csv('data/master.csv')

In [5]:
dups = master.duplicated('Comment')
data = master[-dups]

In [6]:
#data.to_csv('no_dups.csv')

In [7]:
len(data)

88191

In [ ]:
hilary_sim = data[data['Candidate'] == 'Hilary']
sanders_sim = data[data['Candidate'] == 'Sanders']

In [ ]:
biden_sim = data[data['Candidate'] == 'Biden']
trump_sim = data[data['Candidate'] == 'Trump']
bush_sim = data[data['Candidate'] == 'Bush']
carson_sim = data[data['Candidate'] == 'Carson']

In [ ]:
hilary_sim.to_csv('hilary_sim.csv')
sanders_sim.to_csv('sanders_sim.csv')
biden_sim.to_csv('biden_sim.csv')
trump_sim.to_csv('trump_sim.csv')
bush_sim.to_csv('bush_sim.csv')
carson_sim.to_csv('carson_sim.csv')

In [49]:
def word_count(text):
    out = text.translate(string.maketrans("",""), string.punctuation)
    return len(out.split())

In [50]:
data['word_count'] = data['Comment'].apply(lambda x: word_count(str(x)))

/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [51]:
data['Recommendations'] = data['Recommendations'].apply(lambda x: 0 if x == np.nan else x)
x = data['Recommendations'].values
np.percentile(x, 95)


/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


84.0

In [52]:
data['Rec'] = data['Recommendations'].apply(lambda x: 1 if x > 79 else 0)
data['Rec'].value_counts()

/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0    54559
1     3020
dtype: int64

In [53]:
data['Candidate'].value_counts()

Hilary     24982
Trump      11679
Bush        6839
Sanders     6459
Carson      6372
Biden       1248
dtype: int64

In [54]:
cols_encode = ['State', 'Candidate']
encoders = {}
for col in cols_encode:
    le = preprocessing.LabelEncoder()
    le.fit(data[col])
    encoders[col] = le
    data[col] = le.transform(data[col])


/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
reg = data[['EditorPick', 'Sentiment', 'Subjectivity', 'word_count', 'Rec', 'State', 'Candidate']]

In [25]:
# recs = data['Rec'].values
# recs = np.nan_to_num(recs)
# plt.hist(recs, bins=300)


In [24]:
X = reg.drop('Rec', axis = 1).values
X = np.nan_to_num(X)
y = reg['Rec'].values

In [26]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)

In [30]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
y_pred = rf.fit(X_train, y_train).predict(X_test)

print 'Recall: %f'%(metrics.recall_score(y_test, y_pred))
print 'Precision: %f'%(metrics.precision_score(y_test, y_pred))

Recall: 0.530558
Precision: 0.820548


In [21]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

svc_linear = SVC()
y_pred = svc_linear.fit(X_train, y_train).predict(X_test)

print 'Recall: %f'%(metrics.recall_score(y_test, y_pred))
print 'Precision: %f'%(metrics.precision_score(y_test, y_pred))

Recall: 0.000000
Precision: 0.000000


In [65]:
commentOnly = data[['Comment', 'Rec']]

In [107]:
X = commentOnly['Comment']
y = commentOnly['Rec'].values
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)

In [108]:
#type(commentOnly.loc[0,'Comment'])

In [112]:
vectorizer = TfidfVectorizer(stop_words = 'english')

vectorized_X_train  = vectorizer.fit_transform(X_train)
vectorized_X_test = vectorizer.transform(X_test)

In [ ]:
print vectorized_X_train.shape
print y_train.shape

(98263, 64621)
(98263,)


In [56]:
X = reg.drop('Candidate', axis = 1).values
X = np.nan_to_num(X)
y = reg['Candidate'].values

In [57]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)

In [60]:
weights = dict()
for i in range(6):
    weights[i] = x[i]/float(x.sum())

In [61]:
weights

{0: 0.0,
 1: 8.0904054263967272e-07,
 2: 0.0,
 3: 8.0904054263967272e-07,
 4: 3.2361621705586909e-06,
 5: 0.0}

In [59]:
from sklearn.multiclass import OneVsRestClassifier
base_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, class_weight = weights)
ovr = OneVsRestClassifier(estimator=base_clf, n_jobs=-1)
y_pred = ovr.fit(X_train, y_train).predict(X_test)

print 'Accuracy: %f'%(metrics.accuracy_score(y_test, y_pred))
print 'Recall: %f'%(metrics.recall_score(y_test, y_pred))
print 'Precision: %f'%(metrics.precision_score(y_test, y_pred))

An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (519, 0))
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (519, 0))
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (519, 0))
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (519, 0))




An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement', (519, 0))
An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line statement'

JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/Users/datascientist/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/Users/datascientist/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

.............

In [34]:
x = reg['Candidate'].value_counts()

In [99]:
data_drop = data.drop_duplicates('Comment')
len(data_drop)

88191

In [65]:
weights

{0: 0.027510801516621108,
 1: 0.15075831055462482,
 2: 0.14046380389736354,
 3: 0.28143461775857509,
 4: 0.14238162419539724,
 5: 0.2574508420774182}

In [ ]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in comment.lower().split() if word not in stoplist]
        for comment in content]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in texts]

In [ ]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=5)
model.print_topics(5)